In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import os
from pyfaidx import Fasta
from sklearn.metrics import confusion_matrix
from pangolin.model import Pangolin
from pangolin.utils import sequence_to_onehot, rev_comp

# Import hàm của bạn
from metrics import compute_metrics

# --- CẤU HÌNH ---
FASTA_PATH = "hg38.fa" 
MODEL_WEIGHTS = "final.1.0.3.v2"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CONTEXT_WINDOW = 15000
HALF_WINDOW = CONTEXT_WINDOW // 2

TEST_FILES = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']

# --- KHỞI TẠO MODEL ---
genome = Fasta(FASTA_PATH)
model = Pangolin(L=CONTEXT_WINDOW)
model.load_state_dict(torch.load(MODEL_WEIGHTS, map_location=DEVICE))
model.to(DEVICE).eval()

def get_sequence(chrom, pos, strand):
    chrom_key = chrom if chrom.startswith("chr") else f"chr{chrom}"
    start, end = pos - HALF_WINDOW, pos + HALF_WINDOW
    seq = genome[chrom_key][start:end].seq.upper()
    return rev_comp(seq) if strand == '-' else seq

def run_benchmark():
    for file_name in TEST_FILES:
        if not os.path.exists(file_name): continue
            
        print(f"Đang xử lý: {file_name}...")
        df = pd.read_csv(file_name)
        y_true = df['Splicing_types'].values
        
        preds_list = []
        probs_list = []

        for _, row in df.iterrows():
            parts = row['id'].split('_')
            full_seq = get_sequence(parts[1], int(parts[2]), parts[3])
            seq_tensor = sequence_to_onehot(full_seq).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                p_d, p_a = model(seq_tensor)
                
                # Lấy xác suất cao nhất giữa các mô (tissues) tại vị trí trung tâm
                score_d = torch.max(p_d[0, HALF_WINDOW, :]).item()
                score_a = torch.max(p_a[0, HALF_WINDOW, :]).item()

                # Tính toán xác suất 3 lớp
                score_n = max(0, 1 - (score_d + score_a))
                total = score_n + score_d + score_a
                prob_triple = [score_n/total, score_d/total, score_a/total]
                
                probs_list.append(prob_triple)
                preds_list.append(np.argmax(prob_triple))

        y_pred = np.array(preds_list)
        probs = np.array(probs_list)

        # 1. Tính toán metrics cơ bản từ hàm của bạn
        metrics_results = compute_metrics(y_true, y_pred, probs)
        
        # 2. Tính toán Confusion Matrix cho 3 lớp
        # Labels=[0, 1, 2] đảm bảo thứ tự ma trận là Neither, Donor, Acceptor
        cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])
        
        # 3. Thêm vào dict kết quả (Convert sang list để lưu JSON)
        metrics_results['confusion_matrix'] = cm.tolist()
        metrics_results['labels_order'] = ['Neither', 'Donor', 'Acceptor']

        # Lưu kết quả
        output_name = file_name.replace('.csv', '_results.json')
        with open(output_name, 'w', encoding='utf-8') as f:
            json.dump(metrics_results, f, indent=4, ensure_ascii=False)
            
        print(f"Đã lưu kết quả vào {output_name}")